In [1]:
import sys
sys.path.insert(0,'..')
%load_ext autoreload
%autoreload 2

In [9]:
from src.utils import model_utils as mod_utils
from src.utils import twitter_utils as tt_utils
import pandas as pd
import swifter
import numpy as np
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.externals import joblib
from sklearn.preprocessing import LabelEncoder
from keras.models import load_model

In [3]:
df = pd.read_csv("../data/processed/tweets_ambev.csv", sep=';', encoding='utf-8')
df.head(3)

,Unnamed: 0,user,tweet,location
0,0,xuuuuuuuuuuulia,"Feliz dia dos namorados @BrahmaCerveja, @skol,...","Maringá, Brasil"
1,1,PedroPawlowski,O quanto gosto de cada bebida:\n\ngin: 4/10\nv...,"São Paulo, Brasilien"
2,2,rogerjagielski,O quanto gosto de cada bebida:\n\ngin: 0/10 \n...,"Blumenau, Brasil"


In [4]:
X = df['tweet'].values
X

array(['Feliz dia dos namorados @BrahmaCerveja, @skol, @Eisenbahn 😍❤',
       'O quanto gosto de cada bebida:\n\ngin: 4/10\nvodka: 5/10\npinga: 0/10\nvinho: 3/10\ncerveja: 8/10\nwhisky: 4/10\ntequila: 2… https://t.co/8UE6poNxTX',
       'O quanto gosto de cada bebida:\n\ngin: 0/10 \nvodka: 7/10\npinga: 6/10\nvinho: 7/10\ncerveja: 10/10\nwhisky: 10/10\ntequila… https://t.co/VEaGdJx2X2',
       ..., 'Skol não. Skol jamais!',
       '@Tatadebh Obrigada, amiga \nMeu buquê era uma latinha de brahma\nMeu pai perdeu meu casamento pq encheu a cara de quentão\nEsse dia foi lindo',
       'RT @Itspedrito: MEU DEUS SKOL BEATS CONTE COMIGO PRA TUDO https://t.co/XQUUe85cNA'],
      dtype=object)

In [5]:
X = np.array([tt_utils.remove_url(tweet) for tweet in X])
X

array(['Feliz dia dos namorados BrahmaCerveja skol Eisenbahn',
       'O quanto gosto de cada bebidagin 410vodka 510pinga 010vinho 310cerveja 810whisky 410tequila 2',
       'O quanto gosto de cada bebidagin 010 vodka 710pinga 610vinho 710cerveja 1010whisky 1010tequila',
       ..., 'Skol no Skol jamais',
       'Tatadebh Obrigada amiga Meu buqu era uma latinha de brahmaMeu pai perdeu meu casamento pq encheu a cara de quentoEsse dia foi lindo',
       'RT Itspedrito MEU DEUS SKOL BEATS CONTE COMIGO PRA TUDO'],
      dtype='<U137')

In [6]:
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(X)
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_x = t.texts_to_sequences(X)
print(encoded_x)


[[73, 7, 15, 16, 93, 1, 1156], [4, 28, 22, 2, 24, 41, 442, 209, 53, 583, 261, 342, 311], [4, 28, 22, 2, 24, 41, 121, 61, 165, 251, 202, 47, 137], [278, 10, 1157, 2, 77, 10, 1, 60, 4, 36, 95, 1410], [12, 387, 473, 505, 18, 279, 376, 154, 11, 210, 65, 154, 286, 210, 252, 3, 506, 210, 388], [61, 89, 196, 353, 47, 273, 744, 1825, 1826, 1158], [1, 8, 2765, 1159, 98, 623], [12, 120, 7, 15, 16, 30, 29, 6, 13, 26, 9, 25, 5, 17, 19, 21, 3, 27, 20, 34, 35, 40, 18], [12, 243, 36, 99, 1, 42, 240, 146, 9, 100], [4, 62, 6, 2766, 166, 1, 42, 31, 36, 1160, 3, 556, 244, 1161, 2767, 32, 1827, 3, 1162, 33, 232], [7, 15, 16, 30, 29, 6, 13, 26, 9, 25, 5, 17, 19, 21, 3, 27, 20, 34, 35, 10], [4, 28, 22, 2, 24, 41, 102, 167, 53, 155, 47, 85], [1, 42, 146, 2768], [624, 151, 424, 18, 1828, 2769, 1, 42, 23, 1829, 2770, 1163, 62, 6, 90, 171, 474, 365], [407, 104, 323, 1, 78, 245, 32, 94, 63, 181, 396, 33, 408, 8, 49, 6, 2771, 188, 32, 532, 6, 62], [1, 1411, 14, 1412, 456], [1, 78, 5, 14, 625, 246, 33, 36, 214, 37

In [7]:
max_length = 20
padded_x = pad_sequences(encoded_x, maxlen=max_length, padding='post')
print(padded_x)

[[  73    7   15 ...    0    0    0]
 [   4   28   22 ...    0    0    0]
 [   4   28   22 ...    0    0    0]
 ...
 [   1   14    1 ...    0    0    0]
 [  36 1531  124 ...    7  211  228]
 [  12  243   36 ...    0    0    0]]


In [10]:
model = load_model("../models/keras/sentiment-analysis.keras")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [11]:
encoder = joblib.load("../models/encoders/sentiment-label-encoder.pkl")

In [12]:
y_pred = encoder.inverse_transform(np.argmax(model.predict(padded_x)), axis=1)

InternalError: Blas GEMM launch failed : a.shape=(32, 32), b.shape=(32, 32), m=32, n=32, k=32
	 [[{{node gru_5/while/MatMul_4}}]]
	 [[{{node dense_26/Sigmoid}}]]